In [1]:
import itertools
import sys

from collections import defaultdict, deque
from typing import List

### Операции и их порядок

In [2]:
OPERATORS = {"|", ".", "*", "+", "?"}
OP_PRECEDENCE = {
    "|": 1,
    ".": 2,
    "*": 3,
    "+": 3,
    "?": 3,
}

### Токенизация регулярок

In [3]:
def add_concat(regex: str) -> list:
    tokens = []
    i = 0
    while i < len(regex):
        if regex[i] != "\\":
            tokens.append(regex[i])
            i += 1
        else:
            # we have one or more backslashes
            j = i
            while i < len(regex) and regex[i] == "\\":
                i += 1
            n = i - j
            pairs = n // 2
            for _ in range(pairs):
                # represent a literal backslash by "\\"
                tokens.append("\\\\")
            if n % 2 == 1:
                # odd count! last backslash escapes next character
                if i < len(regex):
                    tokens.append("\\" + regex[i])
                    i += 1
                else:
                    # next character does not exist!? treat as literal backslash
                    tokens.append("\\\\")

    def is_literal(tok: str) -> bool:
        # escaped tokens (start with backslash) are literals
        if isinstance(tok, str) and tok.startswith("\\"):
            return True
        # tokens that are operators or parentheses are not literals
        if tok in OPERATORS or tok in {"(", ")"}:
            return False
        return True

    out_tokens = []
    for idx, tok in enumerate(tokens):
        out_tokens.append(tok)
        if idx + 1 >= len(tokens):
            continue
        nxt = tokens[idx + 1]
        # left can be end of an atom if it is a literal, a closing parenthesis, or a postfix operator (* + ?)
        left_can_end = is_literal(tok) or tok == ")" or tok in {"*", "+", "?"}
        # next can be start of an atom if it is a literal or an opening parenthesis
        next_can_start = is_literal(nxt) or nxt == "("
        if left_can_end and next_can_start:
            out_tokens.append(".")

    return out_tokens

In [4]:
add_concat("a(b|c)+\\")

['a', '.', '(', 'b', '|', 'c', ')', '+', '.', '\\\\']

### Reverse Polish Notation

In [5]:
def to_postfix(regex: str) -> str:
    tokens = add_concat(regex)
    output = []
    stack = []
    for tok in tokens:
        # escaped literal
        if tok.startswith("\\"):
            output.append(tok)
        elif tok == "(":
            stack.append(tok)
        elif tok == ")":
            while stack and stack[-1] != "(":
                output.append(stack.pop())
            if not stack:
                raise ValueError("Mismatched parenthesis")
            stack.pop()
        elif tok in OPERATORS:
            # unary postfix operators (* + ?), handle precedence
            while stack and stack[-1] != "(" and (
                OP_PRECEDENCE[stack[-1]] > OP_PRECEDENCE[tok] or
                (OP_PRECEDENCE[stack[-1]] == OP_PRECEDENCE[tok] and tok not in {"*", "+", "?"})
            ):
                output.append(stack.pop())
            stack.append(tok)
        else:
            # literal
            output.append(tok)
    while stack:
        op = stack.pop()
        if op in "()":
            raise ValueError("Mismatched parenthesis")
        output.append(op)
    return " ".join(output)

In [6]:
to_postfix("a(b|c)+\\")

'a b c | + . \\\\ .'

In [7]:
to_postfix("(a|b)*abb")

'a b | * a . b . b .'

Токены

`["a", ".", "(", "b", "|", "c", ")", "+", ".", "\?"]`


tok=`a` → обычный символ → `output = ["a"]`, `stack = []`

tok=`.` → оператор → `stack = ["."]`

tok=`(` → добавляем в стек → `stack = [".", "("]`

tok=`b` → обычный символ → `output = ["a", "b"]`

tok=`|` → оператор → `stack = [".", "(", "|"]`

tok=`c` → обычный символ → `output = ["a", "b", "c"]`

tok=`)` → переносим содержимое скобок из стека в output → `output = ["a", "b","c", "|"]`, `stack = ["."]`

tok=`+` → оператор, приоритет выше чем у `"."`, убираем из стека → `stack = [".", "+"]`

tok=`.` → оператор, приоритет ниже чем у `"+"` → убираем `"+"` из стека → `output = ["a", "b", "c", "|", "+"], stack = ["."]`, теперь верхний элемент в стеке такой же, как новый токен, кладем в output → `output = ["a", "b", "c", "|", "+", "."]`; кладем `"."` в стек → `stack = ["."]`

tok=`\?` → обычный символ → `output = ["a", "b", "c", "|", "+", ".", "\?"]`

в конце смердживаем стек в конец output
`["a", "b", "c", "|", "+", ".", "\?", "."]` превращается в строку `a b c | + . \? .`

### Класс для состояния

In [8]:
class State:
    _ids = itertools.count()
    def __init__(self):
        self.id = next(State._ids)
        # transitions: symbol -> set(states)
        self.transitions = defaultdict(set)
    def add(self, sym, st):
        # sym is character or None for epsilon
        self.transitions[sym].add(st)
    def __repr__(self):
        return f"S{self.id}"

### Класс для недетерминированного конечного автомата (граф из State)

In [9]:
class NFA:
    def __init__(self, start: State, accept: State):
        self.start = start
        self.accept = accept
        self.states = set()
        self._gather()

    def _gather(self):
        # BFS to collect states
        q = deque([self.start])
        seen = set([self.start])
        while q:
            s = q.popleft()
            self.states.add(s)
            for targets in s.transitions.values():
                for t in targets:
                    if t not in seen:
                        seen.add(t)
                        q.append(t)

    @staticmethod
    def single(symbol):
        # creates the simplest possible NFA for a single symbol
        s = State()
        a = State()
        s.add(symbol, a)
        return NFA(s, a)

    def __repr__(self):
        return f"NFA(start={self.start}, accept={self.accept}, states={len(self.states)})"

### Построение НКА

In [10]:
def build_nfa_from_postfix(postfix: str) -> NFA:
    tokens = postfix.split()
    stack = []
    for tok in tokens:
        if tok.startswith("\\"):
            if len(tok) == 1:
                literal = "\\"
            else:
                literal = tok[1]
            stack.append(NFA.single(literal))
        elif tok not in OPERATORS:
            stack.append(NFA.single(tok))
        else:
            if tok == ".":
                # concatenation. connect a.accept -> b.start (via epsilon)
                b = stack.pop()
                a = stack.pop()
                a.accept.add(None, b.start)
                n = NFA(a.start, b.accept)
                stack.append(n)
            elif tok == "|":
                b = stack.pop()
                a = stack.pop()
                s = State()
                f = State()
                s.add(None, a.start)
                s.add(None, b.start)
                a.accept.add(None, f)
                b.accept.add(None, f)
                stack.append(NFA(s, f))
            elif tok == "*":
                a = stack.pop()
                s = State()
                f = State()
                s.add(None, a.start)
                s.add(None, f)
                a.accept.add(None, a.start)
                a.accept.add(None, f)
                stack.append(NFA(s, f))
            elif tok == "+":
                a = stack.pop()
                # A+ = A . A*
                s = State()
                f = State()
                s.add(None, a.start)
                a.accept.add(None, a.start)
                a.accept.add(None, f)
                stack.append(NFA(s, f))
            elif tok == "?":
                a = stack.pop()
                s = State()
                f = State()
                s.add(None, a.start)
                s.add(None, f)
                a.accept.add(None, f)
                stack.append(NFA(s, f))
            else:
                raise ValueError(f"Unknown operator {tok}")
    if len(stack) != 1:
        raise ValueError("Invalid regex: stack leftover")
    return stack[0]

### Избавление от е-дуг

In [11]:
class EpsilonFreeNFA:
    def __init__(self, start, accepts, transitions, alphabet, states):
        self.start = start
        self.accepts = set(accepts)
        self.transitions = transitions  # dict state -> dict symbol -> set(states)
        self.alphabet = set(alphabet)
        self.states = set(states)

In [12]:
def epsilon_closure(states):
    # states: iterable of State objects
    stack = list(states)
    res = set(stack)
    while stack:
        s = stack.pop()
        for t in s.transitions.get(None, []):
            if t not in res:
                res.add(t); stack.append(t)
    return res

In [13]:
def remove_epsilon(nfa: NFA):
    # Build mapping state -> closure using original NFA
    closures = {s: epsilon_closure([s]) for s in nfa.states}
    # Determine alphabet (exclude None)
    alphabet = set()
    for s in nfa.states:
        for sym in s.transitions.keys():
            if sym is not None:
                alphabet.add(sym)
    # New transitions: for each state p and symbol a,
    # targets = union over q in closure(p) of transitions on a from q, then take closure of each target
    new_transitions = {s: defaultdict(set) for s in nfa.states}
    for p in nfa.states:
        for a in alphabet:
            targets = set()
            for q in closures[p]:
                for t in q.transitions.get(a, []):
                    targets.update(closures[t])
            if targets:
                new_transitions[p][a] = targets
    # New accept states: any state p such that closure(p) contains original accept
    new_accepts = set()
    for p in nfa.states:
        if nfa.accept in closures[p]:
            new_accepts.add(p)

    return EpsilonFreeNFA(nfa.start, new_accepts, new_transitions, alphabet, nfa.states)

### Класс для ДКА

In [14]:
class DFA:
    def __init__(self, start, accepts, transitions, alphabet):
        self.start = start
        self.accepts = set(accepts)
        self.transitions = transitions
        self.alphabet = alphabet
        self.states = set(transitions.keys())

### Переход от НКА к ДКА

In [15]:
def nfa_to_dfa(ef_nfa):
    # State in DFA is frozenset of NFA states
    start_set = frozenset([ef_nfa.start])

    queue = deque([start_set])
    dfa_states = {start_set: 0}
    dfa_transitions = {}
    dfa_accepts = set()
    alph = sorted(list(ef_nfa.alphabet))
    while queue:
        S = queue.popleft()
        sid = dfa_states[S]
        dfa_transitions[sid] = {}
        # if any NFA state in S is accepting -> DFA accepting
        if any(s in ef_nfa.accepts for s in S):
            dfa_accepts.add(sid)
        for a in alph:
            # move: for each nfa state in S, union ef_nfa.transitions[state].get(a, [])
            dest = set()
            for p in S:
                dest.update(ef_nfa.transitions.get(p, {}).get(a, set()))
            dest = frozenset(dest)
            if not dest:
                continue
            if dest not in dfa_states:
                dfa_states[dest] = len(dfa_states)
                queue.append(dest)
            dfa_transitions[sid][a] = dfa_states[dest]

    return DFA(0, dfa_accepts, dfa_transitions, alph)

### Оптимизация ДКА

In [16]:
def minimize_dfa(dfa):
    # states are ints 0..n-1
    states = set(dfa.transitions.keys())
    alphabet = list(dfa.alphabet)
    # initialize partitions: accepting and non-accepting
    P = [set(dfa.accepts), states - set(dfa.accepts)]
    P = [s for s in P if s]  # remove empty
    # worklist
    W = deque(P.copy())
    inv = defaultdict(lambda: defaultdict(set))  # inv[a][q] = set of p that transition on a to q
    for p in states:
        for a in alphabet:
            q = dfa.transitions.get(p, {}).get(a, None)
            if q is not None:
                inv[a][q].add(p)
    while W:
        A = W.popleft()
        for a in alphabet:
            # X = states that have transitions on a to A
            X = set()
            for q in A:
                X.update(inv[a].get(q, set()))
            if not X:
                continue
            newP = []
            for Y in P:
                inter = Y & X
                diff = Y - X
                if inter and diff:
                    newP.append(inter)
                    newP.append(diff)
                    # replace Y in P by inter and diff
                    P = [p_ for p_ in P if p_ is not Y]
                    P.extend([inter, diff])
                    # adjust worklist
                    if Y in W:
                        W.remove(Y)
                        W.append(inter); W.append(diff)
                    else:
                        # add smaller
                        W.append(inter if len(inter) <= len(diff) else diff)
                    break
            # continue looping over P until stable

    # Now P contains partition blocks; build new DFA
    # map old state -> block id
    block_of = {}
    for i,block in enumerate(P):
        for s in block:
            block_of[s] = i
    new_start = block_of[dfa.start]
    new_accepts = set()
    for b_idx, block in enumerate(P):
        if any(s in dfa.accepts for s in block):
            new_accepts.add(b_idx)
    new_transitions = {}
    for b_idx, block in enumerate(P):
        # choose representative
        rep = next(iter(block))
        new_transitions[b_idx] = {}
        for a in alphabet:
            q = dfa.transitions.get(rep, {}).get(a, None)
            if q is not None:
                new_transitions[b_idx][a] = block_of[q]

    return DFA(new_start, new_accepts, new_transitions, alphabet)

In [17]:
def match_dfa(dfa, s: str) -> bool:
    cur = dfa.start
    for ch in s:
        if ch not in dfa.alphabet:
            return False
        cur = dfa.transitions.get(cur, {}).get(ch, None)
        if cur is None:
            return False
    return cur in dfa.accepts

### Принты

In [18]:
def pretty_print_nfa(nfa: NFA):
    print("NFA states:", len(nfa.states))
    for s in sorted(nfa.states, key=lambda x: x.id):
        for sym, targets in s.transitions.items():
            for t in targets:
                print(f"  {s} -[{sym if sym is not None else "ε"}]-> {t}")
    print("Start:", nfa.start, "Accept:", nfa.accept)


def pretty_print_efnfa(ef):
    print("Epsilon-free NFA states:", len(ef.states))
    for s in sorted(ef.states, key=lambda x: x.id):
        for a, targets in ef.transitions.get(s, {}).items():
            for t in targets:
                print(f"  {s} -[{a}]-> {t}")
    print("Start:", ef.start, "Accepts:", ef.accepts)


def pretty_print_dfa(dfa):
    print("DFA states:", len(dfa.transitions))
    for s in sorted(dfa.transitions.keys()):
        for a, t in dfa.transitions[s].items():
            print(f"  D{s} -[{a}]-> D{t}")
    print("Start:", dfa.start, "Accepts:", dfa.accepts)

### Проверка

In [19]:
def compile_regex(regex: str):
    postfix = to_postfix(regex)
    nfa = build_nfa_from_postfix(postfix)
    ef = remove_epsilon(nfa)
    dfa = nfa_to_dfa(ef)
    min_dfa = minimize_dfa(dfa)
    return {
        "postfix": postfix,
        "nfa": nfa,
        "ef_nfa": ef,
        "dfa": dfa,
        "min_dfa": min_dfa,
    }

In [20]:
def validate_regex(regex: str, statements: List[str]):
    compiled = compile_regex(regex)
    print("Постфикс:", compiled["postfix"])
    print("NFA states:", len(compiled["nfa"].states))
    print("Epsilon-free NFA states:", len(compiled["ef_nfa"].states))
    print("DFA states (before minimization):", len(compiled["dfa"].transitions))
    print("DFA states (after minimization):", len(compiled["min_dfa"].transitions))
    print("\n--- NFA ---")
    pretty_print_nfa(compiled["nfa"])
    print("\n--- Epsilon-free NFA ---")
    pretty_print_efnfa(compiled["ef_nfa"])
    print("\n--- DFA ---")
    pretty_print_dfa(compiled["dfa"])
    print("\n--- Minimized DFA ---")
    pretty_print_dfa(compiled["min_dfa"])
    print("\nValidating examples\n")
    for statement in statements:
        print(statement)
        ok = match_dfa(compiled["min_dfa"], statement)
        print("ACCEPT" if ok else "REJECT")
        print()

In [21]:
validate_regex(
    regex="a(b|c)+\\",
    statements=[
        "abc",           # REJECT
        "ab\\",          # ACCEPT
        "ac\\",          # ACCEPT
        "a\\",           # REJECT
        "abc\\",         # ACCEPT
        "abcbcbcbb\\",   # ACCEPT
        "b\\"            # REJECT
    ]
)

Постфикс: a b c | + . \\ .
NFA states: 12
Epsilon-free NFA states: 12
DFA states (before minimization): 5
DFA states (after minimization): 4

--- NFA ---
NFA states: 12
  S0 -[a]-> S1
  S1 -[ε]-> S8
  S2 -[b]-> S3
  S3 -[ε]-> S7
  S4 -[c]-> S5
  S5 -[ε]-> S7
  S6 -[ε]-> S4
  S6 -[ε]-> S2
  S7 -[ε]-> S9
  S7 -[ε]-> S6
  S8 -[ε]-> S6
  S9 -[ε]-> S10
  S10 -[\]-> S11
Start: S0 Accept: S11

--- Epsilon-free NFA ---
Epsilon-free NFA states: 12
  S0 -[a]-> S4
  S0 -[a]-> S8
  S0 -[a]-> S1
  S0 -[a]-> S2
  S0 -[a]-> S6
  S1 -[c]-> S4
  S1 -[c]-> S9
  S1 -[c]-> S5
  S1 -[c]-> S10
  S1 -[c]-> S2
  S1 -[c]-> S7
  S1 -[c]-> S6
  S1 -[b]-> S4
  S1 -[b]-> S3
  S1 -[b]-> S9
  S1 -[b]-> S7
  S1 -[b]-> S10
  S1 -[b]-> S2
  S1 -[b]-> S6
  S2 -[b]-> S4
  S2 -[b]-> S3
  S2 -[b]-> S9
  S2 -[b]-> S7
  S2 -[b]-> S10
  S2 -[b]-> S2
  S2 -[b]-> S6
  S3 -[c]-> S4
  S3 -[c]-> S9
  S3 -[c]-> S5
  S3 -[c]-> S10
  S3 -[c]-> S2
  S3 -[c]-> S7
  S3 -[c]-> S6
  S3 -[\]-> S11
  S3 -[b]-> S4
  S3 -[b]-> S3
  S3 -[b]-> 

In [22]:
validate_regex(
    regex=r"(ab\?c)+d?",
    statements=[
        "ab?c",          # ACCEPT
        "ab?cab?c",      # ACCEPT
        "ab?cd",         # ACCEPT
        "ab?cab?cd",     # ACCEPT
        "abc",           # REJECT
        "ab??c",         # REJECT
        "ab?cab?cdd",    # REJECT
        "d",             # REJECT
    ]
)


Постфикс: a b . \? . c . + d ? .
NFA states: 14
Epsilon-free NFA states: 14
DFA states (before minimization): 6
DFA states (after minimization): 6

--- NFA ---
NFA states: 14
  S12 -[a]-> S13
  S13 -[ε]-> S14
  S14 -[b]-> S15
  S15 -[ε]-> S16
  S16 -[?]-> S17
  S17 -[ε]-> S18
  S18 -[c]-> S19
  S19 -[ε]-> S12
  S19 -[ε]-> S21
  S20 -[ε]-> S12
  S21 -[ε]-> S24
  S22 -[d]-> S23
  S23 -[ε]-> S25
  S24 -[ε]-> S25
  S24 -[ε]-> S22
Start: S20 Accept: S25

--- Epsilon-free NFA ---
Epsilon-free NFA states: 14
  S12 -[a]-> S13
  S12 -[a]-> S14
  S13 -[b]-> S16
  S13 -[b]-> S15
  S14 -[b]-> S16
  S14 -[b]-> S15
  S15 -[?]-> S17
  S15 -[?]-> S18
  S16 -[?]-> S17
  S16 -[?]-> S18
  S17 -[c]-> S22
  S17 -[c]-> S21
  S17 -[c]-> S25
  S17 -[c]-> S12
  S17 -[c]-> S19
  S17 -[c]-> S24
  S18 -[c]-> S22
  S18 -[c]-> S21
  S18 -[c]-> S25
  S18 -[c]-> S12
  S18 -[c]-> S19
  S18 -[c]-> S24
  S19 -[d]-> S25
  S19 -[d]-> S23
  S19 -[a]-> S13
  S19 -[a]-> S14
  S20 -[a]-> S13
  S20 -[a]-> S14
  S21 -[d]-> S25
